(ipykernel)
conda activate emotalk
conda install ipykernel
python -m ipykernel install --user --name emotalk --display-name "emotalk (Python 3.8.8)"

## Get names of shape keys  

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("shape_keys_data.csv")
print(df.head())

  shape_key_name  weight  vertex_index         x          y         z
0          Basis     0.0             0  0.112536  15.373697  8.076443
1          Basis     0.0             1 -0.010624  20.928453  2.844734
2          Basis     0.0             2  0.027110  19.293489  5.438893
3          Basis     0.0             3  0.048687  13.813027  8.882374
4          Basis     0.0             4  0.016785  11.369778  9.257272


In [3]:
first_column = df.iloc[:, 0]
unique_values = first_column.drop_duplicates()

print(unique_values)

0                       Basis
6191             browDownLeft
12382           browDownRight
18573             browInnerUp
24764         browOuterUpLeft
30955        browOuterUpRight
37146               cheekPuff
43337         cheekSquintLeft
49528        cheekSquintRight
55719            eyeBlinkLeft
61910           eyeBlinkRight
68101         eyeLookDownLeft
74292        eyeLookDownRight
80483           eyeLookInLeft
86674          eyeLookInRight
92865          eyeLookOutLeft
99056         eyeLookOutRight
105247          eyeLookUpLeft
111438         eyeLookUpRight
117629          eyeSquintLeft
123820         eyeSquintRight
130011            eyeWideLeft
136202           eyeWideRight
142393             jawForward
148584                jawLeft
154775                jawOpen
160966               jawRight
167157             mouthClose
173348        mouthDimpleLeft
179539       mouthDimpleRight
185730         mouthFrownLeft
191921        mouthFrownRight
198112            mouthFunnel
204303    

## blendshape_to_3d

In [4]:
import numpy as np
blendshape_file = './result/disgust.npy'
blendshape_data = np.load(blendshape_file)

print(blendshape_data.shape)
print(blendshape_data[:])

(104, 52)
[[ 4.80752110e-01  4.83027428e-01  4.41512018e-02 ...  2.69665092e-01
   2.72663623e-01  3.50964977e-03]
 [ 5.06709039e-01  5.09331107e-01  4.21265177e-02 ...  2.83665121e-01
   2.85715848e-01  4.25785175e-03]
 [ 5.21364510e-01  5.23877501e-01  4.11513187e-02 ...  2.92902768e-01
   2.93683112e-01  4.75363852e-03]
 ...
 [ 6.12852573e-01  6.14181757e-01  4.17615138e-02 ...  2.87572503e-01
   2.86453336e-01 -1.15992501e-03]
 [ 6.04408264e-01  6.06112540e-01  4.41922955e-02 ...  2.87779957e-01
   2.88186878e-01 -9.20026214e-04]
 [ 5.91184497e-01  5.93407035e-01  4.73454781e-02 ...  2.83585340e-01
   2.85654485e-01 -3.73832765e-04]]


In [5]:
!./blender/blender ./render.blend --background --python /transfer/emotalk/EmoTalk_release-main/blendshape_to_3d.py

Blender 3.4.1 (hash 55485cb379f7 built 2022-12-20 00:46:45)
Read prefs: /home/s5727214/.config/blender/3.4/config/userpref.blend
Read blend: /transfer/emotalk/EmoTalk_release-main/render.blend
Error: Python: Traceback (most recent call last):
  File "/transfer/emotalk/EmoTalk_release-main/blendshape_to_3d.py", line 91, in <module>
    convert_blendshapes_to_vertices(input_path, output_path, obj_name="face")
  File "/transfer/emotalk/EmoTalk_release-main/blendshape_to_3d.py", line 37, in convert_blendshapes_to_vertices
    blendshape_coeffs = np.load(input_path)
  File "/transfer/emotalk/EmoTalk_release-main/blender/3.4/python/lib/python3.10/site-packages/numpy/lib/npyio.py", line 407, in load
    fid = stack.enter_context(open(os_fspath(file), "rb"))
FileNotFoundError: [Errno 2] No such file or directory: '--python'

Blender quit


## extract face data

In [3]:
!./blender/blender ./render.blend --background --python /transfer/emotalk/EmoTalk_release-main/experiment/extract_blender_face_data.py

Blender 3.4.1 (hash 55485cb379f7 built 2022-12-20 00:46:45)
Read prefs: /home/s5727214/.config/blender/3.4/config/userpref.blend
Read blend: /transfer/emotalk/EmoTalk_release-main/render.blend
复制对象 'face_tri' 已创建并三角化
Saved face_tri.npy with shape: (12346, 3)

Blender quit


## visualize the 3d data（trimesh）

In [4]:
import numpy as np
import trimesh
import pyrender

# 加载 3D 顶点数据
vertices_all = np.load('./result/3d_result/disgust.npy')
print("Vertices shape:", vertices_all.shape)  # 例如 (104, 5023, 3)# 示例：取第 50 帧的顶点数据，并使用 trimesh 和 pyrender 可视化
frame = 50
vertices = vertices_all[frame]  # shape (5023, 3)

# 假设你的面部网格拓扑信息（faces）是已知的，
# 这里我们假设 faces 数组已保存或从其他地方加载（通常 FLAME 模型会提供）
faces = np.load('./result/3d_result/faces_tri.npy')  # 示例：faces 的形状 (num_faces, 3)

# 创建 trimesh 对象
tri_mesh = trimesh.Trimesh(vertices, faces)
mesh = pyrender.Mesh.from_trimesh(tri_mesh)

# 创建场景并添加网格
scene = pyrender.Scene()
scene.add(mesh)

# 可视化
pyrender.Viewer(scene, use_raymond_lighting=True)

Vertices shape: (104, 6191, 3)


Viewer(width=640, height=480)

In [12]:
from util_eval import visualize_frame

vertices_file = "./result/3d_result/disgust.npy"
frame_index = 40
visualize_frame(vertices_file,frame_index)

Vertices shape: (104, 6191, 3)


In [ ]:
#可视化当前帧模型
vertices_file = "./result/3d_result/01-01-08-02-02-02-01_vertices.npy"
frame_index = 50
visualize_frame(vertices_file,frame_index)

## 3D-ETF dataset

In [4]:
import numpy as np

blendshape_file = '../3D-ETF/HDTF/blendshape/RD_Radio1_000.npy'
blendshape_data = np.load(blendshape_file)

print(blendshape_data.shape)
print(blendshape_data[:])

(2249, 52)
[[0.02709011 0.0255003  0.25166535 ... 0.06851197 0.06823228 0.00317504]
 [0.03119782 0.03131485 0.28532061 ... 0.0613993  0.06313494 0.00130084]
 [0.01891743 0.02003305 0.29329833 ... 0.06667861 0.06713656 0.00057581]
 ...
 [0.03378208 0.03240672 0.40738258 ... 0.09349071 0.08554596 0.00062552]
 [0.03633351 0.03235664 0.37267575 ... 0.09088102 0.0833602  0.00062583]
 [0.03293969 0.02904696 0.36840293 ... 0.08063286 0.07738558 0.00064074]]


#### translate ground truth to 3d data 

In [ ]:
# translate ground truth to 3d data 
import os

# Blender 可执行文件路径
blender_exe = "/transfer/emotalk/EmoTalk_release-main/blender/blender"

# .blend 文件路径（需要包含 "face" 对象）
blend_file = "/transfer/emotalk/EmoTalk_release-main/render.blend"

# 转换脚本路径
script_path = "/transfer/emotalk/EmoTalk_release-main/blendshape_to_3d.py"

# 输入 blendshape 参数文件（例如从 HDTF 或其他数据集）
input_file = "/transfer/emotalk/3D-ETF/LVE_test/blendshape/01-01-08-02-02-02-01.npy"

# 输出 3D 顶点文件路径
output_file = "/transfer/emotalk/3D-ETF/LVE_test/3d_result/01-01-08-02-02-02-01.npy"

# 构造 Blender 命令（注意 -- 后面的参数会传给脚本）
command = f'{blender_exe} "{blend_file}" --background --python "{script_path}" -- "{input_file}" "{output_file}"'
print("Executing command:", command)
os.system(command)
# translate predicted to 3d data 
!python demo.py --wav_path "/transfer/emotalk/3D-ETF/LVE_test/audio/01-01-08-02-02-02-01.wav"
#predicted data
import os

# Blender 可执行文件路径
blender_exe = "/transfer/emotalk/EmoTalk_release-main/blender/blender"

# .blend 文件路径（需要包含 "face" 对象）
blend_file = "/transfer/emotalk/EmoTalk_release-main/render.blend"

# 转换脚本路径
script_path = "/transfer/emotalk/EmoTalk_release-main/blendshape_to_3d.py"

# 输入 blendshape 参数文件（例如从 HDTF 或其他数据集）
input_file = "/transfer/emotalk/EmoTalk_release-main/result/01-01-08-02-02-02-01.npy"

# 输出 3D 顶点文件路径
output_file = "/transfer/emotalk/EmoTalk_release-main/result/3d_result/01-01-08-02-02-02-01.npy"

# 构造 Blender 命令（注意 -- 后面的参数会传给脚本）
command = f'{blender_exe} "{blend_file}" --background --python "{script_path}" -- "{input_file}" "{output_file}"'
print("Executing command:", command)
os.system(command)

In [ ]:
import numpy as np

# Load the converted 3D vertices file
vertices = np.load("./result/3d_result/disgust.npy")

# Print the shape of the vertices array
print("Shape of vertices:", vertices.shape)

# Print a sample of vertices from the first frame (e.g., first 10 vertices)
print("Sample vertices:", vertices[30:50][:10])


#### translate predicted to 3d data 

In [6]:
# translate predicted to 3d data 
!python demo.py --wav_path "/transfer/emotalk/3D-ETF/LVE_test/audio/01-01-08-02-02-02-01.wav"
#predicted data
import os

# Blender 可执行文件路径
blender_exe = "/transfer/emotalk/EmoTalk_release-main/blender/blender"

# .blend 文件路径（需要包含 "face" 对象）
blend_file = "/transfer/emotalk/EmoTalk_release-main/render.blend"

# 转换脚本路径
script_path = "/transfer/emotalk/EmoTalk_release-main/blendshape_to_3d.py"

# 输入 blendshape 参数文件（例如从 HDTF 或其他数据集）
input_file = "/transfer/emotalk/EmoTalk_release-main/result/01-01-08-02-02-02-01.npy"

# 输出 3D 顶点文件路径
output_file = "/transfer/emotalk/EmoTalk_release-main/result/3d_result/01-01-08-02-02-02-01.npy"

# 构造 Blender 命令（注意 -- 后面的参数会传给脚本）
command = f'{blender_exe} "{blend_file}" --background --python "{script_path}" -- "{input_file}" "{output_file}"'
print("Executing command:", command)
os.system(command)


config.json: 100%|█████████████████████████| 1.53k/1.53k [00:00<00:00, 24.6MB/s]
config.json: 100%|█████████████████████████| 1.53k/1.53k [00:00<00:00, 22.0MB/s]
model.safetensors: 100%|████████████████████| 1.26G/1.26G [00:11<00:00, 114MB/s]
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-english and are newly initialized: ['wav2vec2.lm_head.bias', 'wav2vec2.lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of Wav2Vec2ForSpeechClassification were not initialized from the model checkpoint at r-f/wav2vec-english-speech-emotion-recognition and are newly initialized: ['wav2vec2.lm_head.bias', 'wav2vec2.lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/transfer/emotalk/EmoTalk_release-main/demo.py:21: FutureWarning: You are using `torch.load` with `weigh

0

In [9]:
vertices_file_GT = '../3D-ETF/LVE_test/3d_result/RD_Radio1_000_vertices.npy'
vertices_data_GT = np.load(blendshape_file)

print(vertices_data_GT.shape)
print(vertices_data_GT[:])

(2249, 52)
[[0.02709011 0.0255003  0.25166535 ... 0.06851197 0.06823228 0.00317504]
 [0.03119782 0.03131485 0.28532061 ... 0.0613993  0.06313494 0.00130084]
 [0.01891743 0.02003305 0.29329833 ... 0.06667861 0.06713656 0.00057581]
 ...
 [0.03378208 0.03240672 0.40738258 ... 0.09349071 0.08554596 0.00062552]
 [0.03633351 0.03235664 0.37267575 ... 0.09088102 0.0833602  0.00062583]
 [0.03293969 0.02904696 0.36840293 ... 0.08063286 0.07738558 0.00064074]]


## Evaluate model

In [7]:
import numpy as np
np.set_printoptions(precision=10)

def compute_lve_frame(pred_vertices, gt_vertices, lip_indices, mode='max'):
    """
    计算单帧中嘴部区域的顶点误差。
    
    参数：
      pred_vertices: (N, 3) 的预测顶点数组
      gt_vertices:   (N, 3) 的真实顶点数组
      lip_indices:   一个列表或数组，包含嘴部区域顶点的索引
      mode:          'max' 取最大误差，'mean' 取平均误差
      
    返回：
      当前帧的 LVE（单位通常为毫米）
    """
    # 取出嘴部区域的顶点
    pred_lips = pred_vertices[lip_indices]
    gt_lips = gt_vertices[lip_indices]
    # 计算欧氏距离（ℓ2 范数）——每个顶点的误差
    errors = np.linalg.norm(pred_lips - gt_lips, axis=1)
    if mode == 'max':
        return np.max(errors)
    elif mode == 'mean':
        return np.mean(errors)
    else:
        raise ValueError("mode 参数必须是 'max' 或 'mean'")

def compute_average_lve(pred_all, gt_all, lip_indices, mode='max'):
    
    """
    对整个视频序列计算平均的 LVE。
    
    参数：
      pred_all: (F, N, 3) 预测顶点数组，F 为帧数
      gt_all:   (F, N, 3) 真实顶点数组
      lip_indices: 嘴部区域顶点的索引列表
      mode: 'max' 或 'mean'
      
    返回：
      整个视频序列的平均 LVE
    """
    # 取预测和 ground truth 中最小的帧数
    num_frames_pred = pred_all.shape[0]
    num_frames_gt = gt_all.shape[0]
    num_frames = min(num_frames_pred, num_frames_gt)
    
    # 确保 lip_indices 中的每个索引都小于当前帧顶点数量
    N = pred_all[0].shape[0]
    lip_indices = [i for i in lip_indices if i < N]
    
    lve_list = []
    for f in range(num_frames):
        frame_lve = compute_lve_frame(pred_all[f], gt_all[f], lip_indices, mode)
        lve_list.append(frame_lve)
    return np.mean(lve_list)

# 示例使用：

# 1. 加载预测的 3D 顶点数据和 ground truth 数据（.npy 文件）
# 这里假设文件名分别为 "predicted_vertices.npy" 和 "ground_truth_vertices.npy"
pred_vertices_all = np.load("/transfer/emotalk/EmoTalk_release-main/result/3d_result/01-01-08-02-02-02-01.npy")  # shape: (F, N, 3)
gt_vertices_all = np.load('../3D-ETF/LVE_test/3d_result/01-01-08-02-02-02-01.npy')  # shape: (F, N, 3)
print("Predicted vertices shape:", pred_vertices_all.shape)
print("Ground truth vertices shape:", gt_vertices_all.shape)

# 2. 定义嘴部区域顶点索引
# 这里你可以通过文本文件加载，或直接在代码中定义一个列表，例如：
# 假设嘴部区域的顶点索引为：
lip_indices = [9, 10, 11, 18, 25, 26, 34, 35, 36, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 120, 127, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 148, 151, 153, 155, 419, 420, 433, 434, 435, 436, 470, 485, 486, 487, 488, 489, 490, 527, 528, 529, 530, 531, 532, 537, 538, 544, 545, 549, 553, 554, 555, 560, 563, 564, 569, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 588, 589, 591, 597, 598, 599, 601, 602, 608, 609, 610, 611, 614, 637, 638, 639, 640, 641, 642, 644, 648, 693, 694, 695, 697, 751, 754, 755, 756, 757, 758, 765, 766, 767, 910, 913, 914, 929, 1093, 1095, 1096, 1097, 1110, 1126, 1127, 1128, 1129, 1131, 1155, 1156, 1157, 1158, 1161, 1162, 1163, 1164, 1165, 1166, 1167, 1168, 1169, 1170, 1171, 1172, 1173, 1174, 1175, 1176, 1182, 1231, 1232, 1236, 1237, 1380, 1381, 1388, 1463, 1464, 1465, 1487, 1645, 1646, 1647, 1660, 1661, 1662, 1671, 1672, 1673, 1674, 1675, 1676, 1677, 1678, 1679, 1680, 1681, 1682, 1683, 1684, 1685, 1686, 1687, 1688, 1689, 1690, 1691, 1692, 1693, 1694, 1695, 1696, 1697, 1698, 1699, 1700, 1701, 1702, 1703, 1704, 1705, 1706, 1707, 1708, 1709, 1710, 1711, 1712, 1713, 1714, 1715, 1716, 1717, 1718, 1719, 1720, 1721, 1722, 1723, 1724, 1725, 1726, 1727, 1728, 1729, 1730, 1731, 1732, 1733, 1734, 1735, 1736, 1737, 1738, 1739, 1740, 1741, 1742, 1743, 1744, 1745, 1746, 1747, 1748, 1749, 1750, 1751, 1752, 1753, 1754, 1755, 1756, 1757, 1758, 1759, 1760, 1761, 1762, 1763, 1764, 1765, 1766, 1767, 1768, 1769, 1770, 1771, 1772, 1773, 1774, 1775, 1776, 1777, 1778, 1779, 1780, 1781, 1782, 1783, 1784, 1785, 1786, 1787, 1788, 1789, 1790, 1791, 1792, 1793, 1794, 1795, 1796, 1797, 1798, 1799, 1800, 1801, 1802, 1803, 1804, 1805, 1806, 1807, 1808, 1809, 1810, 1811, 1812, 1813, 1814, 1815, 1816, 1817, 1818, 1819, 1820, 1821, 1822, 1823, 1824, 1825, 1826, 1827, 1828, 1829, 1830, 1831, 1832, 1833, 1834, 1835, 1836, 1837, 1838, 1839, 1840, 1841, 1842, 1843, 1844, 1845, 1846, 1847, 1848, 1849, 1850, 1851, 1852, 1853, 1854, 1855, 1856, 1857, 1858, 1859, 1860, 1861, 1862, 1863, 1864, 1865, 1866, 1867, 1868, 1869, 1870, 1871, 1872, 1873, 1874, 1875, 1876, 1877, 1878, 1879, 1880, 1881, 1882, 1883, 1884, 1885, 1886, 1887, 1888, 1889, 1890, 1891, 1892, 1893, 1894, 1895, 1896, 1897, 1898, 1899, 1900, 1901, 1902, 1903, 1904, 1905, 1906, 1907, 1908, 1909, 1910, 1911, 1912, 1913, 1914, 1915, 1916, 1917, 1918, 1919, 1920, 1921, 1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939, 1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052, 2053, 2054, 2110, 2111, 2164, 2236, 2255, 2256, 2257, 2260, 2263, 2264, 2265, 2443, 2444, 2445, 2446, 2447, 2448, 2449, 2450, 2451, 2452, 2453, 2454, 2455, 2456, 2457, 2458, 2459, 2465, 2466, 2467, 2468, 2469, 2470, 2471, 2472, 2473, 2474, 2475, 2476, 2477, 2478, 2479, 2480, 2481, 2482, 2483, 2484, 2485, 2486, 2487, 2488, 2489, 2490, 2491, 2492, 2493, 2494, 2495, 2496, 2497, 2498, 2499, 2500, 2501, 2502, 2503, 2504, 2505, 2506, 2507, 2508, 2509, 2510, 2511, 2512, 2513, 2514, 2515, 2516, 2517, 2518, 2519, 2520, 2521, 2522, 2523, 2524, 2525, 2526, 2527, 2528, 2529, 2530, 2531, 2532, 2533, 2534, 2535, 2536, 2537, 2538, 2539, 2540, 2541, 2542, 2543, 2544, 2545, 2546, 2547, 2548, 2561, 2562, 2563, 2564, 2565, 2566, 2567, 2578, 2579, 2592, 2593, 2594, 2595, 2596, 2597, 2598, 2601, 2606, 2607, 2608, 2609, 2610, 2611, 2612, 2613, 2614, 2615, 2616, 2617, 2618, 2619, 2626, 2627, 2628, 2629, 2630, 2631, 2632, 2645, 2646, 2647, 2648, 2649, 2650, 2651, 2652, 2653, 2654, 2655, 2656, 2657, 2658, 2659, 2660, 2661, 2662, 2670, 2671, 2672, 2673, 2674, 2675, 2676, 2677, 2678, 2679, 2680, 2681, 2682, 2683, 2684, 2685, 2686, 2696, 2697, 2698, 2699, 2700, 2701, 2702, 2703, 2704, 2705, 2706, 2820, 2821, 2822, 2922, 2923, 2924, 2925, 2926, 2927, 2928, 2929, 2930, 2931, 2932, 2933, 2934, 2935, 2936, 2937, 2938, 2939, 2940, 2941, 2942, 2943, 2944, 2945, 2946, 2947, 2948, 2949, 2950, 2951, 2952, 2953, 2954, 2955, 2956, 2957, 2958, 2959, 2960, 2961, 2962, 2963, 2964, 2965, 2966, 2967, 2968, 2969, 2970, 2971, 2972, 2973, 2974, 2975, 2976, 2977, 2978, 2979, 2980, 2981, 2982, 2983, 2984, 2985, 2986, 2987, 2988, 2989, 2990, 2991, 2992, 2993, 2996, 2997, 2998, 2999, 3000, 3001, 3002, 3003, 3004, 3005, 3006, 3007, 3008, 3009, 3011, 3012, 3013, 3014, 3015, 3016, 3017, 3018, 3019, 3020, 3021, 3022, 3023, 3024, 3025, 3026, 3027, 3029, 3030, 3031, 3032, 3033, 3034, 3035, 3036, 3037, 3038, 3039, 3040, 3041, 3042, 3043, 3044, 3045, 3046, 3049, 3050, 3051, 3052, 3053, 3054, 3055, 3056, 3057, 3064, 3065, 3066, 3067, 3068, 3069, 3070, 3071, 3072, 3073, 3074, 3075, 3076, 3077, 3078, 3079, 3080, 3081, 3082, 3115, 3116, 3117, 3118, 3119, 3120, 3121, 3122, 3143, 3144, 3145, 3146, 3147, 3148, 3149, 3150, 3151, 3441, 3442, 3455, 3456, 3457, 3458, 3492, 3507, 3508, 3509, 3510, 3511, 3512, 3549, 3550, 3551, 3552, 3553, 3554, 3559, 3560, 3566, 3567, 3571, 3575, 3576, 3577, 3582, 3585, 3586, 3591, 3594, 3595, 3596, 3597, 3598, 3599, 3600, 3601, 3602, 3603, 3604, 3605, 3606, 3610, 3611, 3613, 3619, 3620, 3621, 3623, 3624, 3630, 3631, 3632, 3633, 3636, 3659, 3660, 3661, 3662, 3663, 3664, 3666, 3670, 3715, 3716, 3717, 3718, 3719, 3773, 3776, 3777, 3778, 3779, 3780, 3787, 3788, 3789, 3932, 3935, 3936, 3951, 4115, 4117, 4118, 4119, 4132, 4148, 4149, 4150, 4151, 4153, 4177, 4178, 4179, 4180, 4183, 4184, 4185, 4186, 4187, 4188, 4189, 4190, 4191, 4192, 4193, 4194, 4195, 4196, 4197, 4198, 4204, 4253, 4254, 4258, 4259, 4402, 4403, 4410, 4485, 4486, 4487, 4509, 4510, 4512, 4513, 4667, 4668, 4669, 4683, 4684, 4693, 4694, 4695, 4696, 4697, 4698, 4699, 4700, 4701, 4702, 4703, 4704, 4705, 4706, 4707, 4708, 4709, 4710, 4711, 4712, 4713, 4714, 4715, 4716, 4717, 4718, 4719, 4720, 4721, 4722, 4723, 4724, 4725, 4726, 4727, 4728, 4729, 4730, 4731, 4732, 4733, 4734, 4735, 4736, 4737, 4738, 4739, 4740, 4741, 4742, 4743, 4744, 4745, 4746, 4747, 4748, 4749, 4750, 4751, 4752, 4753, 4754, 4755, 4756, 4757, 4758, 4759, 4760, 4761, 4762, 4763, 4764, 4765, 4766, 4767, 4768, 4769, 4770, 4771, 4772, 4773, 4774, 4775, 4776, 4777, 4778, 4779, 4780, 4781, 4782, 4783, 4784, 4785, 4786, 4787, 4788, 4789, 4790, 4791, 4792, 4793, 4794, 4795, 4796, 4797, 4798, 4799, 4800, 4801, 4802, 4803, 4804, 4805, 4806, 4807, 4808, 4809, 4810, 4811, 4812, 4813, 4814, 4815, 4816, 4817, 4818, 4819, 4820, 4821, 4822, 4823, 4824, 4825, 4826, 4827, 4828, 4829, 4830, 4831, 4832, 4833, 4834, 4835, 4836, 4837, 4838, 4839, 4840, 4841, 4842, 4843, 4844, 4845, 4846, 4847, 4848, 4849, 4850, 4851, 4852, 4853, 4854, 4855, 4856, 4857, 4858, 4859, 4860, 4861, 4862, 4863, 4864, 4865, 4866, 4867, 4868, 4869, 4870, 4871, 4872, 4873, 4874, 4875, 4876, 4877, 4878, 4879, 4880, 4881, 4882, 4883, 4884, 4885, 4886, 4887, 4888, 4889, 4890, 4891, 4892, 4893, 4894, 4895, 4896, 4897, 4898, 4899, 4900, 4901, 4902, 4903, 4904, 4905, 4906, 4907, 4908, 4909, 4910, 4911, 4912, 4913, 4914, 4915, 4916, 4917, 4918, 4919, 4920, 4921, 4922, 4923, 4924, 4925, 4926, 4927, 4928, 4929, 4930, 4931, 4932, 4933, 4934, 4935, 4936, 4937, 4938, 4939, 4940, 4941, 4942, 4943, 4944, 4945, 4946, 4947, 4948, 4949, 4950, 4951, 4952, 4953, 4954, 4955, 4956, 4957, 4958, 4959, 4960, 4961, 4962, 4963, 4964, 4965, 4966, 4967, 4968, 4969, 4970, 4971, 4972, 4973, 4974, 4975, 4976, 4977, 4978, 4979, 4980, 4981, 4982, 4983, 4984, 4985, 4986, 4987, 4988, 4989, 4990, 4991, 4992, 4993, 4994, 4995, 4996, 4997, 4998, 4999, 5000, 5001, 5002, 5003, 5004, 5005, 5006, 5007, 5008, 5009, 5010, 5011, 5012, 5013, 5014, 5015, 5016, 5017, 5018, 5019, 5020, 5021, 5022, 5023, 5024, 5025, 5026, 5027, 5028, 5029, 5030, 5031, 5032, 5033, 5034, 5035, 5036, 5037, 5038, 5039, 5040, 5041, 5042, 5043, 5044, 5045, 5046, 5047, 5048, 5049, 5050, 5051, 5052, 5053, 5054, 5055, 5056, 5057, 5058, 5059, 5060, 5061, 5062, 5063, 5064, 5065, 5066, 5067, 5068, 5069, 5070, 5071, 5072, 5073, 5074, 5075, 5076, 5132, 5133, 5258, 5277, 5278, 5279, 5282, 5283, 5285, 5286, 5287, 5465, 5466, 5467, 5468, 5469, 5470, 5471, 5472, 5473, 5474, 5475, 5476, 5477, 5478, 5479, 5480, 5481, 5487, 5488, 5489, 5490, 5491, 5492, 5493, 5494, 5495, 5496, 5497, 5498, 5499, 5500, 5501, 5502, 5503, 5504, 5505, 5506, 5507, 5508, 5509, 5510, 5511, 5512, 5513, 5514, 5515, 5516, 5517, 5518, 5519, 5520, 5521, 5522, 5523, 5524, 5525, 5526, 5527, 5528, 5529, 5530, 5531, 5532, 5533, 5534, 5535, 5536, 5537, 5538, 5539, 5540, 5541, 5542, 5543, 5544, 5545, 5546, 5547, 5548, 5549, 5550, 5551, 5552, 5553, 5554, 5555, 5556, 5557, 5558, 5559, 5560, 5561, 5562, 5563, 5564, 5565, 5566, 5567, 5568, 5569, 5570, 5583, 5584, 5585, 5586, 5587, 5588, 5589, 5601, 5614, 5615, 5616, 5617, 5618, 5619, 5620, 5623, 5628, 5629, 5630, 5631, 5632, 5633, 5634, 5635, 5636, 5637, 5638, 5639, 5640, 5641, 5648, 5649, 5650, 5651, 5652, 5653, 5654, 5655, 5656, 5667, 5668, 5669, 5670, 5671, 5672, 5673, 5674, 5675, 5676, 5677, 5678, 5679, 5680, 5681, 5682, 5683, 5684, 5685, 5692, 5693, 5694, 5695, 5696, 5697, 5698, 5699, 5700, 5701, 5702, 5703, 5704, 5705, 5706, 5707, 5708, 5709, 5710, 5718, 5719, 5720, 5721, 5722, 5723, 5724, 5725, 5726, 5727, 5728, 5842, 5843, 5844, 5936, 5937, 5938, 5939, 5940, 5941, 5942, 5943, 5944, 5945, 5946, 5947, 5948, 5949, 5950, 5951, 5952, 5953, 5954, 5955, 5956, 5957, 5958, 5959, 5960, 5961, 5962, 5963, 5964, 5965, 5966, 5967, 5968, 5969, 5970, 5971, 5972, 5973, 5974, 5975, 5976, 5977, 5978, 5979, 5980, 5981, 5982, 5983, 5984, 5985, 5986, 5987, 5988, 5989, 5990, 5991, 5992, 5993, 5994, 5995, 5996, 5997, 5998, 5999, 6000, 6001, 6002, 6003, 6004, 6005, 6006, 6007, 6008, 6010, 6011, 6012, 6013, 6014, 6015, 6016, 6017, 6018, 6019, 6020, 6021, 6022, 6023, 6024, 6025, 6026, 6027, 6028, 6029, 6030, 6031, 6032, 6033, 6034, 6035, 6036, 6037, 6038, 6039, 6040, 6041, 6042, 6043, 6044, 6045, 6046, 6047, 6048, 6049, 6050, 6051, 6052, 6053, 6054, 6055, 6056, 6057, 6058, 6059, 6060, 6063, 6064, 6065, 6066, 6067, 6068, 6069, 6070, 6071, 6077, 6078, 6079, 6080, 6081, 6082, 6083, 6084, 6085, 6086, 6087, 6088, 6089, 6090, 6091, 6092, 6093, 6094, 6095, 6122, 6127, 6128, 6129, 6130, 6131, 6132, 6133, 6134, 6155, 6156, 6157, 6158, 6159, 6160, 6161, 6162, 6163]  # 示例数据，请根据你的模型实际情况设置

# 3. 计算平均 LVE（使用最大误差作为每帧的指标，这里 mode='max'）
average_lve = compute_average_lve(pred_vertices_all, gt_vertices_all, lip_indices, mode='max')
print("Average Lip Vertex Error (LVE):", average_lve)


Predicted vertices shape: (99, 6191, 3)
Ground truth vertices shape: (98, 6191, 3)
Average Lip Vertex Error (LVE): 0.0


In [15]:
frame = 300  # 或其他帧
pred_frame = pred_vertices_all[frame]
gt_frame = gt_vertices_all[frame]
# 提取嘴部区域顶点
pred_lips = pred_frame[lip_indices]
gt_lips = gt_frame[lip_indices]
# 计算每个顶点的误差
errors = np.linalg.norm(pred_lips - gt_lips, axis=1)
print("该帧嘴部最大误差：", np.max(errors))
print("该帧嘴部平均误差：", np.mean(errors))


该帧嘴部最大误差： 0.0
该帧嘴部平均误差： 0.0


In [2]:
import numpy as np
blendshape_file = '/transfer/emotalk/EmoTalk_release-main/result/RD_Radio1_000.npy'
blendshape_data = np.load(blendshape_file)

print(blendshape_data.shape)
print(blendshape_data[:])

(2250, 52)
[[0.02860429 0.02946426 0.31636077 ... 0.0683267  0.0695871  0.00255811]
 [0.02551783 0.02594968 0.31741223 ... 0.06508837 0.06652807 0.00247168]
 [0.02339699 0.0233254  0.32034895 ... 0.06392843 0.06495434 0.00273694]
 ...
 [0.04431644 0.04176137 0.47903994 ... 0.07221591 0.06859898 0.00593734]
 [0.04651878 0.0438632  0.46705773 ... 0.07296525 0.06925234 0.00599154]
 [0.05093047 0.04819831 0.44567326 ... 0.07312731 0.06898379 0.00611499]]


In [19]:
blendshape_file = '/transfer/emotalk/3D-ETF/LVE_test/3d_result/RD_Radio1_000_vertices.npy'
blendshape_data = np.load(blendshape_file)

print(blendshape_data.shape)
print(blendshape_data[:])

(2249, 6191, 3)
[[[ 1.12536013e-01  1.53736973e+01  8.07644272e+00]
  [-1.06239999e-02  2.09284534e+01  2.84473395e+00]
  [ 2.71099806e-02  1.92934895e+01  5.43889284e+00]
  ...
  [-1.79152000e+00  7.29602098e+00  8.71479702e+00]
  [-1.89661396e+00  7.39863777e+00  8.64548111e+00]
  [-2.02000999e+00  7.52041006e+00  8.58892441e+00]]

 [[ 1.12536013e-01  1.53736973e+01  8.07644272e+00]
  [-1.06239999e-02  2.09284534e+01  2.84473395e+00]
  [ 2.71099806e-02  1.92934895e+01  5.43889284e+00]
  ...
  [-1.79152000e+00  7.29602098e+00  8.71479702e+00]
  [-1.89661396e+00  7.39863777e+00  8.64548111e+00]
  [-2.02000999e+00  7.52041006e+00  8.58892441e+00]]

 [[ 1.12536013e-01  1.53736973e+01  8.07644272e+00]
  [-1.06239999e-02  2.09284534e+01  2.84473395e+00]
  [ 2.71099806e-02  1.92934895e+01  5.43889284e+00]
  ...
  [-1.79152000e+00  7.29602098e+00  8.71479702e+00]
  [-1.89661396e+00  7.39863777e+00  8.64548111e+00]
  [-2.02000999e+00  7.52041006e+00  8.58892441e+00]]

 ...

 [[ 1.12536013e-01

In [8]:
value = 0.000000123456789
print(f"{value:.10f}")

0.0000001235


## bs to 3d错误，test2里得到修正